<a href="https://colab.research.google.com/github/550tealeaves/DATA-70500-working-with-data/blob/main/HW4_LinearModels2.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Linear Models, part II: Logistic Regression

We'll use the global social indicators data to develop a logistic regression model and pratice interpreting the results.

First, we'll import the libraries we'll need for this model.


In [1]:
import pandas as pd
import numpy as np
import statsmodels.api as sm
import seaborn as sb
import math


Here's some information about where these social indicators were created: https://hdr.undp.org/data-center/composite-indices


Next, we'll read in the data sources and create the DataFrame with all of our variables.

In [2]:
#Load datasets and use na_values to note missing values
Employees = pd.read_csv('https://raw.githubusercontent.com/550tealeaves/DATA-70500-working-with-data/refs/heads/main/datasets/Employee.csv', index_col='EmployeeID', na_values=[np.nan])
Employees.head()

,FirstName,LastName,Gender,Age,BusinessTravel,Department,DistanceFromHome (KM),State,Ethnicity,Education,...,MaritalStatus,Salary,StockOptionLevel,OverTime,HireDate,Attrition,YearsAtCompany,YearsInMostRecentRole,YearsSinceLastPromotion,YearsWithCurrManager
EmployeeID,,,,,,,,,,,,,,,,,,,,,
3012-1A41,Leonelle,Simco,Female,30,Some Travel,Sales,27,IL,White,5,...,Divorced,102059,1,No,2012-01-03,No,10,4,9,7
CBCB-9C9D,Leonerd,Aland,Male,38,Some Travel,Sales,23,CA,White,4,...,Single,157718,0,Yes,2012-01-04,No,10,6,10,0
95D7-1CE9,Ahmed,Sykes,Male,43,Some Travel,Human Resources,29,CA,Asian or Asian American,4,...,Married,309964,1,No,2012-01-04,No,10,6,10,8
47A0-559B,Ermentrude,Berrie,Non-Binary,39,Some Travel,Technology,12,IL,White,3,...,Married,293132,0,No,2012-01-05,No,10,10,10,0
42CC-040A,Stace,Savege,Female,29,Some Travel,Human Resources,29,CA,White,2,...,Single,49606,0,No,2012-01-05,Yes,6,1,1,6


In [9]:
#Remove rows with duplicate indices so pd.concat works
Employees = Employees.loc[~Employees.index.duplicated(keep='first')]

In [3]:
PerformanceRating = pd.read_csv('https://raw.githubusercontent.com/550tealeaves/DATA-70500-working-with-data/refs/heads/main/datasets/PerformanceRating.csv', index_col='EmployeeID', na_values=[np.nan])
PerformanceRating.head()

,PerformanceID,ReviewDate,EnvironmentSatisfaction,JobSatisfaction,RelationshipSatisfaction,TrainingOpportunitiesWithinYear,TrainingOpportunitiesTaken,WorkLifeBalance,SelfRating,ManagerRating
EmployeeID,,,,,,,,,,
79F7-78EC,PR01,1/2/2013,5,4,5,1,0,4,4,4
B61E-0F26,PR02,1/3/2013,5,4,4,1,3,4,4,3
F5E3-48BB,PR03,1/3/2013,3,4,5,3,2,3,5,4
0678-748A,PR04,1/4/2013,5,3,2,2,0,2,3,2
541F-3E19,PR05,1/4/2013,5,2,3,1,0,4,4,3


In [10]:
#Remove rows with duplicate indices so pd.concat works
PerformanceRating = PerformanceRating.loc[~PerformanceRating.index.duplicated(keep='first')]

In [4]:
# ThirdChance = pd.read_csv('https://raw.githubusercontent.com/550tealeaves/DATA-70500-working-with-data/refs/heads/main/datasets/third_chance.csv', index_col='outfit.id', na_values=[np.nan])
# ThirdChance.head()

In [12]:
#Use concat method to combine the datasets - all the variables for same outfit id are aligned
#datasets are not stacked, they are merged - index labels the rows
EmployeePerformance = pd.concat([Employees, PerformanceRating], axis=1)
EmployeePerformance.info('verbose')
EmployeePerformance.head()

<class 'pandas.core.frame.DataFrame'>
Index: 1470 entries, 3012-1A41 to 84D4-D4C3
Data columns (total 32 columns):
 #   Column                           Non-Null Count  Dtype  
---  ------                           --------------  -----  
 0   FirstName                        1470 non-null   object 
 1   LastName                         1470 non-null   object 
 2   Gender                           1470 non-null   object 
 3   Age                              1470 non-null   int64  
 4   BusinessTravel                   1470 non-null   object 
 5   Department                       1470 non-null   object 
 6   DistanceFromHome (KM)            1470 non-null   int64  
 7   State                            1470 non-null   object 
 8   Ethnicity                        1470 non-null   object 
 9   Education                        1470 non-null   int64  
 10  EducationField                   1470 non-null   object 
 11  JobRole                          1470 non-null   object 
 12  MaritalStatu

,FirstName,LastName,Gender,Age,BusinessTravel,Department,DistanceFromHome (KM),State,Ethnicity,Education,...,PerformanceID,ReviewDate,EnvironmentSatisfaction,JobSatisfaction,RelationshipSatisfaction,TrainingOpportunitiesWithinYear,TrainingOpportunitiesTaken,WorkLifeBalance,SelfRating,ManagerRating
EmployeeID,,,,,,,,,,,,,,,,,,,,,
3012-1A41,Leonelle,Simco,Female,30,Some Travel,Sales,27,IL,White,5,...,PR1295,10/30/2016,3.0,3.0,2.0,3.0,0.0,4.0,3.0,3.0
CBCB-9C9D,Leonerd,Aland,Male,38,Some Travel,Sales,23,CA,White,4,...,PR1213,7/30/2016,3.0,3.0,2.0,2.0,0.0,5.0,4.0,4.0
95D7-1CE9,Ahmed,Sykes,Male,43,Some Travel,Human Resources,29,CA,Asian or Asian American,4,...,PR1035,4/8/2016,3.0,3.0,2.0,1.0,0.0,2.0,3.0,2.0
47A0-559B,Ermentrude,Berrie,Non-Binary,39,Some Travel,Technology,12,IL,White,3,...,PR1016,3/27/2016,3.0,5.0,3.0,3.0,0.0,2.0,3.0,3.0
42CC-040A,Stace,Savege,Female,29,Some Travel,Human Resources,29,CA,White,2,...,PR1107,5/13/2016,4.0,3.0,2.0,1.0,0.0,5.0,3.0,2.0


Now, we'll compute a binary variable that will be our dependent variable, Y. Then, we'll identify the relevant independent variables and put them in a new DataFrame, X. At that point, we can compute the model.

In [15]:
#Looking at distance from home for employees
EmployeePerformance['DistanceFromHome (KM)'].describe()

,DistanceFromHome (KM)
count,1470.000000
mean,22.502721
std,12.811124
min,1.000000
25%,12.000000
50%,22.000000
75%,33.000000
max,45.000000


In [17]:
#Looking at Employee age - looks to be a relatively young workforce
EmployeePerformance['Age'].describe()

,Age
count,1470.000000
mean,28.989796
std,7.993055
min,18.000000
25%,23.000000
50%,26.000000
75%,34.000000
max,51.000000


In [ ]:
# Turn index into binary variable - compare high GII values & everyone else
# use LOC method to perform a binary test
GlobalIndicatorsTotal['GII Binary'] = 0 #crete new variable
GlobalIndicatorsTotal.loc[GlobalIndicatorsTotal['Gender Inequality Index (GII)'] < 0.19, ['GII Binary']] = 1 #These are nations with a low gender inequality score--that is, the highest gender equality - change values to 1 if they are in the lower quartile
GlobalIndicatorsTotal['GII Binary'].describe() #now see that every country is either 0 or 1

,GII Binary
count,188.000000
mean,0.207447
std,0.406561
min,0.000000
25%,0.000000
50%,0.000000
75%,0.000000
max,1.000000


In [ ]:
#binary variables are typically 0 and 1
GlobalIndicatorsTotal['GII Binary']

,GII Binary
Country,
Norway,1
Australia,1
Switzerland,1
Denmark,1
Netherlands,1
...,...
Burundi,0
Chad,0
Eritrea,0


In [ ]:
#Create linear model to explain probability of being in the high GII group (group coded 1)
# uses 5 potential independent variables
Y = GlobalIndicatorsTotal['GII Binary']
X = GlobalIndicatorsTotal[['Percent Representation in Parliament', 'Population with Secondary Education (Female)', 'Labour Force Participation Rate (Female)', 'Life Expectancy at Birth', 'Gross National Income (GNI) per Capita']]
model0 = sm.Logit(Y, X, missing='drop').fit()
print(model0.summary())

Optimization terminated successfully.
         Current function value: 0.321133
         Iterations 8
                           Logit Regression Results                           
Dep. Variable:             GII Binary   No. Observations:                  156
Model:                          Logit   Df Residuals:                      151
Method:                           MLE   Df Model:                            4
Date:                Wed, 02 Oct 2024   Pseudo R-squ.:                  0.4289
Time:                        00:07:16   Log-Likelihood:                -50.097
converged:                       True   LL-Null:                       -87.724
Covariance Type:            nonrobust   LLR p-value:                 1.760e-15
                                                   coef    std err          z      P>|z|      [0.025      0.975]
----------------------------------------------------------------------------------------------------------------
Percent Representation in Parliament    

- LLR p-value is < 0.05 = model is reliabie (large dataset)
- Pseudo R-squared (goodness of fit) - ~43% of being high GII is accounted for in the model - but pseudo R-squared is not always best way to explain probability
- must convert the coefficients to odds b/c they are not interpretable

- labor fource participation rate & life expectancy at birth have negative odds

In [ ]:
print(math.exp(model0.params[0]), math.exp(model0.params[1]), 1/math.exp(model0.params[2]), 1/math.exp(model0.params[3]), math.exp(1000*model0.params[4]))
# We need to exponentiate (or the take anti-logs of) the coefficients in order to interpret them as odds.
#For the negative coefficients, it is useful to take the inverse of the result & interpret it in the opposite direction (that is, the odds of not being in the high gender equality group). You can also change the increment of change in X
# as is the case here with the parameter for GNI per capita; I changed the increment to $1000 instead of $1.

1.113047941683358 1.0677546839634509 1.0600888084476103 1.086297381643493 1.0540509355600345


<ipython-input-15-1b1cc13454ea>:1: FutureWarning: Series.__getitem__ treating keys as positions is deprecated. In a future version, integer keys will always be treated as labels (consistent with DataFrame behavior). To access a value by position, use `ser.iloc[pos]`
  print(math.exp(model0.params[0]), math.exp(model0.params[1]), 1/math.exp(model0.params[2]), 1/math.exp(model0.params[3]), math.exp(1000*model0.params[4]))


### **Summary**
For each unit increase in high gender equality group, there is
- For each 1% increase in percent representation in Parliament, there is 1.11 times more likely to be in the high equality group
- For each 1% increase in female population with secondary education, there are 1.06 greater odds of being in the high equality group
- For each 1% increase in female force labor participation, there is 1.06 (greater odds) times more likely to be in the low equality group
- For each 1% increase in life expectancy at birth, you are 1.08 times more likely to be in the low equality group.
- For each 1% increase in GNI per capita, it is 1.05 times more likely to be in the high equality group.

In [ ]:
print(np.exp(model0.params)) #These are expressed as odds ratios

Percent Representation in Parliament            1.113048
Population with Secondary Education (Female)    1.067755
Labour Force Participation Rate (Female)        0.943317
Life Expectancy at Birth                        0.920558
Gross National Income (GNI) per Capita          1.000053
dtype: float64


- How much likely is a unit to be high GII with one of these independent variables

In [ ]:
#often show log regression results as marginal effects - not a straight line
model0_marginals = model0.get_margeff() #These are the average effects - aka the avg slope
print(model0_marginals.summary())

        Logit Marginal Effects       
Dep. Variable:             GII Binary
Method:                          dydx
At:                           overall
                                                  dy/dx    std err          z      P>|z|      [0.025      0.975]
----------------------------------------------------------------------------------------------------------------
Percent Representation in Parliament             0.0109      0.002      4.922      0.000       0.007       0.015
Population with Secondary Education (Female)     0.0066      0.001      5.889      0.000       0.004       0.009
Labour Force Participation Rate (Female)        -0.0059      0.002     -2.653      0.008      -0.010      -0.002
Life Expectancy at Birth                        -0.0084      0.001     -6.576      0.000      -0.011      -0.006
Gross National Income (GNI) per Capita        5.335e-06   1.13e-06      4.728      0.000    3.12e-06    7.55e-06


In [ ]:
#changes in odds for predictors at the median - can exponentiate them to convert them into odds
model0_marginals = model0.get_margeff(at='median') #It is often more useful to get estimages of the effect sizes at particular values for the factors
print(model0_marginals.summary())

        Logit Marginal Effects       
Dep. Variable:             GII Binary
Method:                          dydx
At:                            median
                                                  dy/dx    std err          z      P>|z|      [0.025      0.975]
----------------------------------------------------------------------------------------------------------------
Percent Representation in Parliament             0.0055      0.002      2.711      0.007       0.002       0.009
Population with Secondary Education (Female)     0.0034      0.001      3.074      0.002       0.001       0.006
Labour Force Participation Rate (Female)        -0.0030      0.001     -2.608      0.009      -0.005      -0.001
Life Expectancy at Birth                        -0.0043      0.002     -2.647      0.008      -0.007      -0.001
Gross National Income (GNI) per Capita        2.708e-06    1.1e-06      2.458      0.014    5.48e-07    4.87e-06


In [ ]:
mdn_rep = np.exp(0.0055*10)
print("The effect of median representation in parliament is", f"{mdn_rep:.3f} times more likely to be a high quality nation for an increase of ten percent in representation.")

The effect of median representation in parliament is 1.057 times more likely to be a high quality nation for an increase of ten percent in representation.


In [ ]:
model0_pred = model0.pred_table()
print(model0_pred) # Correct predictions are on the diagonal of the 2d array.

[[109.   8.]
 [ 12.  27.]]


In [ ]:
correct_i = 109 / (109 + 8) # The proportion of correct predictions of 0.
correct_j = 27 / (27 + 12) # The proportion of correct predictions of 1.
print(correct_i, correct_j)

0.9316239316239316 0.6923076923076923


## Activity

1. Find and read into a DataFrame a suitable dataset. You may use the global social indictors data from the example here. You may need to combine files, as shown here.

2. Identify a dependent variable to explain. Create a binary variable of this measure, if needed. Explain why you chose this variable or recoded in the way you did.

3. Build a model to explain the DV. You can use the odds (anti-log of the coefficients) or the marginal effects to test for the unique effects of each predictor.

4. Explain the results.